In [148]:
import folium
import pandas as pd
import numpy as np
from haversine import haversine
from tqdm import tqdm

In [149]:
market_region = pd.read_csv('./2019상권데이터 최종.csv', encoding='EUC-KR')
market_region

,연도_분기,상권_코드_명,상권_코드,점포_수,long,lat,분기당_매출_금액,분기당_매출_건수,주말_매출_금액,주말_매출_건수,...,시간대_건수~06_매출_건수,시간대_건수~11_매출_건수,시간대_건수~14_매출_건수,시간대_건수~17_매출_건수,시간대_건수~21_매출_건수,시간대_건수~24_매출_건수,개업_점포_수,폐업_점포_수,유사_업종_점포_수,프랜차이즈_점포_수
0,2019_2분기,이태원 관광특구,1001491,1720,126.994377,37.534435,102083170176,4441275,44766913881,2004774,...,971166,277152,598188,669709,1140530,784530,53,59,1791,71
1,2019_2분기,명동 남대문 북창동 다동 무교동 관광특구,1001492,8631,126.981856,37.564149,392306973005,16655824,84341907910,3388811,...,180088,2385613,5451618,3706300,3891596,1040609,143,227,9143,512
2,2019_2분기,동대문패션타운 관광특구,1001493,13270,127.011025,37.567312,204760984791,6077340,72902937985,1861725,...,595337,574678,1117839,1229527,1674341,885618,103,332,13483,213
3,2019_2분기,종로?청계 관광특구,1001494,5552,126.997374,37.569998,172469071750,6280385,41093828642,1518417,...,225781,708437,1752419,1314164,1629558,650026,120,127,5787,235
4,2019_2분기,잠실 관광특구,1001495,2192,127.115275,37.516476,152732007138,7978513,55835671807,2914005,...,364563,752341,1669688,1743960,2461305,986656,68,93,2391,199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1642,2019_2분기,고분다리전통시장,2130322,128,127.133023,37.542208,4915668137,312544,1340541486,90002,...,9459,31182,49248,63320,121773,37562,6,9,135,7
1643,2019_2분기,둔촌역전통시장,2130323,200,127.134310,37.527818,9494074543,422786,2875077028,126755,...,14175,32752,75634,98389,157803,44033,10,6,209,9
1644,2019_2분기,길동복조리시장,2130324,119,127.143563,37.538925,4607059662,199121,1342266070,56381,...,102,17110,44049,51593,78492,7775,9,6,124,5
1645,2019_2분기,명일전통시장,2130325,96,127.145337,37.549293,3793840824,223786,1122917537,67660,...,568,25102,45467,56783,83982,11884,5,5,101,5


# http://www.hotelrestaurant.co.kr/news/article.html?no=1021
# 상권의 위치별 1차상권은 500m범위로 잡았습니다.

In [150]:
my_map = folium.Map(location=[37.5502, 126.982], zoom_start=11, title="Stamen Toner")

for each in range(len(market_region)):
    marker_circle = folium.Circle([market_region['lat'][each], market_region['long'][each]],
                                       radius = 500,
                                        color = 'skyblue',
                                        popup = market_region['상권_코드_명'][each],
                                        fill_color = 'skyblue'
                                       )
    marker_circle.add_to(my_map)

In [151]:
my_map

# 상권의 규모는 결국 총 유동인구의 수가 중요하다고 한다. 그래 그냥 유동인구로 처리해보자

- 우선 버스, 지하철, 주차장 데이터를 들고오자

- 모든 상권 주변에 유동인구를 구해보자 상권의 범위는 500m로 잡았다

In [152]:
car = pd.read_csv('./park_state.csv')
car_data = car[['위도', '경도', '총 주차면']]
car_data

,위도,경도,총 주차면
0,37.649279,127.040474,71
1,37.643917,127.058567,177
2,37.569939,127.035145,54
3,37.515205,126.915116,98
4,37.525526,126.895794,190
...,...,...,...
1088,37.622745,127.018428,95
1089,37.633227,127.020711,80
1090,37.612180,126.956535,40
1091,37.507334,126.891653,65


In [153]:
# 환승가능한 지하철역은 어떻게 처리할거야?
subway = pd.read_csv('./metro_2019_10_location.csv', encoding = "EUC-KR")
subway


,Unnamed: 0,사용월,호선명,지하철역,출근시간(04~09) 승차,출근시간(07~10) 하차,퇴근시간(16~20) 승차,퇴근시간(17~21) 하차,점심시간,저녁시간,늦은시간퇴근(20~02) 승차,총 승차 인원,총 하차 인원,월,연도,lat,lng
0,0,201910,9호선,흑석,65143,62697,99114,98430,62468,48166,56582,332193,337982,10,2019,37.508770,126.963708
1,1,201910,6호선,효창공원앞,66258,43822,70940,60161,40597,35808,32825,243329,221626,10,2019,37.539261,126.961351
2,2,201910,경의중앙선,효창공원앞,18808,24894,36943,25435,25263,20817,15383,97364,106201,10,2019,37.539261,126.961351
3,3,201910,4호선,회현,40413,305129,467241,91555,385553,185660,37378,996448,1060470,10,2019,37.558514,126.978246
4,4,201910,경원선,회룡,180355,41343,60587,169198,20455,68431,84346,436831,422445,10,2019,37.724846,127.046895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,432,201910,1호선,가산디지털단지,60879,170842,267757,103725,182306,132696,46679,523298,610571,10,2019,37.481072,126.882343
433,433,201910,7호선,가산디지털단지,78866,395297,714564,127277,424169,263525,48660,1229607,1223519,10,2019,37.481072,126.882343
434,434,201910,3호선,가락시장,77365,45275,89474,94395,38756,42075,46568,314088,307058,10,2019,37.492522,127.118234
435,435,201910,8호선,가락시장,41913,53410,80753,76684,55110,46648,33537,250431,282660,10,2019,37.492522,127.118234


In [154]:
'''
# 버스 2022년만
bus = pd.read_csv('./2021_10_bus_data_final.csv')
bus_data = bus[['좌표X', '좌표Y', '총 승차 인원', '총 하차 인원', '환승 가능한 버스의 수']]
bus_data.rename(columns={bus_data.columns[0]: "X좌표"}, inplace=True)
bus_data.rename(columns={bus_data.columns[1]: "Y좌표"}, inplace=True)
'''

'''
# 버스 2021년만
bus = pd.read_csv('./2021_10_bus_data_final.csv')
bus_data = bus[['X좌표', 'Y좌표', '총 승차 인원', '총 하차 인원', '환승 가능한 버스의 수']]
'''


# 버스 2020년만
# 버스 2019년만
bus = pd.read_csv('./2019_10_bus_data_final.csv')
bus_data = bus[['X좌표', 'Y좌표', '총 승차 인원', '총 하차 인원', '환승 가능한 버스의 수']]
bus_data





,X좌표,Y좌표,총 승차 인원,총 하차 인원,환승 가능한 버스의 수
0,126.987786,37.569764,15331,18343,4
1,126.996520,37.579179,82567,72002,17
2,126.998290,37.582709,140393,101718,17
3,126.987507,37.568582,24172,25003,5
4,127.001694,37.586230,54650,95239,16
...,...,...,...,...,...
10515,127.165440,37.560067,482,969,2
10516,127.162906,37.560282,2049,2228,2
10517,127.168974,37.576545,39,2253,1
10518,127.148073,37.559227,2092,12114,2


In [155]:
bus.head()

,연도,월,표준버스정류장ID,역명,환승 가능한 버스의 수,총 승차 인원,총 하차 인원,출근시간(04~09) 승차,출근시간(07~10) 하차,퇴근시간(16~20) 승차,퇴근시간(17~21) 하차,늦은시간(20~00) 승차,늦은시간(21~02) 하차,점심시간,저녁시간,X좌표,Y좌표
0,2019,10,100000001,종로2가사거리,4,15331,18343,4253,3536,2851,4971,2481,2435,4501,5147,126.987786,37.569764
1,2019,10,100000002,창경궁.서울대학교병원,17,82567,72002,22954,36400,23752,6374,9109,1777,34524,24135,126.996520,37.579179
2,2019,10,100000003,명륜3가.성대입구,17,140393,101718,25950,34136,36963,19088,35531,5501,34959,20508,126.998290,37.582709
3,2019,10,100000004,종로2가.삼일교,5,24172,25003,1725,6622,7710,6125,6956,1824,7495,5217,126.987507,37.568582
4,2019,10,100000005,혜화동로터리.여운형활동터,16,54650,95239,17508,26997,11428,22735,7798,5626,33002,36555,127.001694,37.586230


In [156]:
subway_data = subway[['지하철역',  '호선명', '총 승차 인원', '총 하차 인원', 'lat', 'lng']]
subway_data

,지하철역,호선명,총 승차 인원,총 하차 인원,lat,lng
0,흑석,9호선,332193,337982,37.508770,126.963708
1,효창공원앞,6호선,243329,221626,37.539261,126.961351
2,효창공원앞,경의중앙선,97364,106201,37.539261,126.961351
3,회현,4호선,996448,1060470,37.558514,126.978246
4,회룡,경원선,436831,422445,37.724846,127.046895
...,...,...,...,...,...,...
432,가산디지털단지,1호선,523298,610571,37.481072,126.882343
433,가산디지털단지,7호선,1229607,1223519,37.481072,126.882343
434,가락시장,3호선,314088,307058,37.492522,127.118234
435,가락시장,8호선,250431,282660,37.492522,127.118234


# 상권내 모든 주차장에 사람이 1.5명씩 타고 있다 가정
-> 주관적인 판단이라 조금 위험할 수 있다고 생각됨.

In [157]:
market_region_human = market_region[['상권_코드_명', '점포_수', '분기당_매출_금액', '분기당_매출_건수', 'long', 'lat']]
market_region_human['유동인구'] = 0
market_region_human['환승가능한버스노선'] = 0
market_region_human['500m 버스정류장'] = 0

market_region_human['환승가능지하철'] =0
market_region_human['환승가능 호선명'] = ""
market_region_human['500m 지하철역'] = 0
market_region_human

C:\Users\김두엽\AppData\Local\Temp\ipykernel_13368\36209935.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_region_human['유동인구'] = 0
C:\Users\김두엽\AppData\Local\Temp\ipykernel_13368\36209935.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_region_human['환승가능한버스노선'] = 0
C:\Users\김두엽\AppData\Local\Temp\ipykernel_13368\36209935.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

,상권_코드_명,점포_수,분기당_매출_금액,분기당_매출_건수,long,lat,유동인구,환승가능한버스노선,500m 버스정류장,환승가능지하철,환승가능 호선명,500m 지하철역
0,이태원 관광특구,1720,102083170176,4441275,126.994377,37.534435,0,0,0,0,,0
1,명동 남대문 북창동 다동 무교동 관광특구,8631,392306973005,16655824,126.981856,37.564149,0,0,0,0,,0
2,동대문패션타운 관광특구,13270,204760984791,6077340,127.011025,37.567312,0,0,0,0,,0
3,종로?청계 관광특구,5552,172469071750,6280385,126.997374,37.569998,0,0,0,0,,0
4,잠실 관광특구,2192,152732007138,7978513,127.115275,37.516476,0,0,0,0,,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1642,고분다리전통시장,128,4915668137,312544,127.133023,37.542208,0,0,0,0,,0
1643,둔촌역전통시장,200,9494074543,422786,127.134310,37.527818,0,0,0,0,,0
1644,길동복조리시장,119,4607059662,199121,127.143563,37.538925,0,0,0,0,,0
1645,명일전통시장,96,3793840824,223786,127.145337,37.549293,0,0,0,0,,0


In [158]:
market_region_human

,상권_코드_명,점포_수,분기당_매출_금액,분기당_매출_건수,long,lat,유동인구,환승가능한버스노선,500m 버스정류장,환승가능지하철,환승가능 호선명,500m 지하철역
0,이태원 관광특구,1720,102083170176,4441275,126.994377,37.534435,0,0,0,0,,0
1,명동 남대문 북창동 다동 무교동 관광특구,8631,392306973005,16655824,126.981856,37.564149,0,0,0,0,,0
2,동대문패션타운 관광특구,13270,204760984791,6077340,127.011025,37.567312,0,0,0,0,,0
3,종로?청계 관광특구,5552,172469071750,6280385,126.997374,37.569998,0,0,0,0,,0
4,잠실 관광특구,2192,152732007138,7978513,127.115275,37.516476,0,0,0,0,,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1642,고분다리전통시장,128,4915668137,312544,127.133023,37.542208,0,0,0,0,,0
1643,둔촌역전통시장,200,9494074543,422786,127.134310,37.527818,0,0,0,0,,0
1644,길동복조리시장,119,4607059662,199121,127.143563,37.538925,0,0,0,0,,0
1645,명일전통시장,96,3793840824,223786,127.145337,37.549293,0,0,0,0,,0


In [159]:
#37먼저
sub_count = 0
bus_count = 0



for i in tqdm(range(len(market_region_human))):
    sub_count = 0
    bus_count = 0
    
    sub_station = []
    sub_station_name = []
    
    hosun = ""

    market_region_location =  (market_region_human['lat'][i], market_region_human['long'][i])
    
    for each in range(len(bus_data)):
        bus_data_location = (bus_data['Y좌표'][each], bus_data['X좌표'][each])
        a = haversine(market_region_location, bus_data_location, unit = 'm')
        if a < 500:
            bus_count = bus_count + 1
            market_region_human['유동인구'][i] = market_region_human['유동인구'][i] +  bus_data['총 하차 인원'][each]
            market_region_human['환승가능한버스노선'][i] = market_region_human['환승가능한버스노선'][i] +  bus_data['환승 가능한 버스의 수'][each]
    
    market_region_human['환승가능한버스노선'][i] = market_region_human['환승가능한버스노선'][i] / bus_count
    market_region_human['500m 버스정류장'][i] = bus_count

    
    
    for each in range(len(subway_data)):
        subway_data_location = (subway_data['lat'][each], subway_data['lng'][each])
        a = haversine(market_region_location, subway_data_location, unit = 'm')
        if a < 500:
            market_region_human['유동인구'][i] = market_region_human['유동인구'][i] +  subway_data['총 하차 인원'][each]
            sub_station.append(subway_data["호선명"][each])
            
 
                
            if (subway_data["지하철역"][each] not in sub_station_name):
                sub_count = sub_count + 1
                sub_station_name.append(subway_data["지하철역"][each])
                
            sub_station = list(set(sub_station))
    
    for j in range(len(sub_station)):
        hosun = hosun + sub_station[j]
        if j+1 != len(sub_station):
            hosun = hosun + ';'
                   
    market_region_human['환승가능지하철'][i] = len(sub_station)
    market_region_human['500m 지하철역'][i] =  sub_count
    market_region_human['환승가능 호선명'][i] =  hosun

    
    market_region_human['유동인구'][i] = (market_region_human['유동인구'][i]) / 31
    
    
    for each in range(len(car_data)):
        car_data_location = (car_data['위도'][each], car_data['경도'][each])
        a = haversine(market_region_location, car_data_location, unit = 'm')
        if a < 500:
            market_region_human['유동인구'][i] = market_region_human['유동인구'][i] +  (car_data['총 주차면'][each] * 1.5)

  0%|                                                                                         | 0/1647 [00:00<?, ?it/s]C:\Users\김두엽\AppData\Local\Temp\ipykernel_13368\2619123448.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_region_human['유동인구'][i] = market_region_human['유동인구'][i] +  bus_data['총 하차 인원'][each]
C:\Users\김두엽\AppData\Local\Temp\ipykernel_13368\2619123448.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_region_human['환승가능한버스노선'][i] = market_region_human['환승가능한버스노선'][i] +  bus_data['환승 가능한 버스의 수'][each]
C:\Users\김두엽\AppData\Local\Temp\ipykernel_13368\2619123448.py:26: SettingWithCopyW

In [160]:
market_region_human_avr = market_region_human
market_region_human_avr.head(30)

,상권_코드_명,점포_수,분기당_매출_금액,분기당_매출_건수,long,lat,유동인구,환승가능한버스노선,500m 버스정류장,환승가능지하철,환승가능 호선명,500m 지하철역
0,이태원 관광특구,1720,102083170176,4441275,126.994377,37.534435,30909.564516,1.904762,21,1,6호선,1
1,명동 남대문 북창동 다동 무교동 관광특구,8631,392306973005,16655824,126.981856,37.564149,138946.870968,9.555556,18,2,1호선;2호선,2
2,동대문패션타운 관광특구,13270,204760984791,6077340,127.011025,37.567312,107866.096774,5.473684,19,4,5호선;4호선;1호선;2호선,2
3,종로?청계 관광특구,5552,172469071750,6280385,126.997374,37.569998,69533.048387,5.160000,25,3,5호선;1호선;2호선,2
4,잠실 관광특구,2192,152732007138,7978513,127.115275,37.516476,11423.967742,2.500000,8,2,9호선;8호선,2
5,강남 마이스 관광특구,851,173926103805,4768635,127.060059,37.511003,101897.145161,5.818182,22,2,9호선;2호선,2
6,이북5도청사,41,1646731825,81619,126.956884,37.609345,2942.064516,2.312500,16,0,,0
7,독립문역 1번,104,11436343104,595996,126.957209,37.575540,19348.903226,2.954545,22,1,3호선,1
8,세검정초등학교,124,1957855750,135243,126.958801,37.604489,9851.548387,2.281250,32,0,,0
9,대신고등학교,28,926653250,21284,126.960550,37.573289,27390.967742,2.950000,40,1,3호선,1


# 대형상권은 : 하루 유동인구가 10만명이상
# 중형상권은 : 하루 유동인구가 2만명이상

http://www.hotelrestaurant.co.kr/news/article.html?no=3529


In [161]:
big_market_region= market_region_human_avr.loc[market_region_human_avr['유동인구'] >= 100000]
big_market_region

,상권_코드_명,점포_수,분기당_매출_금액,분기당_매출_건수,long,lat,유동인구,환승가능한버스노선,500m 버스정류장,환승가능지하철,환승가능 호선명,500m 지하철역
1,명동 남대문 북창동 다동 무교동 관광특구,8631,392306973005,16655824,126.981856,37.564149,138946.870968,9.555556,18,2,1호선;2호선,2
2,동대문패션타운 관광특구,13270,204760984791,6077340,127.011025,37.567312,107866.096774,5.473684,19,4,5호선;4호선;1호선;2호선,2
5,강남 마이스 관광특구,851,173926103805,4768635,127.060059,37.511003,101897.145161,5.818182,22,2,9호선;2호선,2
79,서울역 12번,224,8431663655,422323,126.974074,37.550903,110658.435484,4.352941,17,4,4호선;1호선;공항철도 1호선;경의중앙선,1
514,연희대우아파트앞,203,6654088324,508368,126.928549,37.560047,132480.064516,9.400000,15,3,2호선;공항철도 1호선;경의중앙선,1
525,경의중앙 신촌역,93,3211881577,176531,126.940548,37.557657,105433.290323,6.266667,30,2,경의중앙선;2호선,2
569,홍대입구역 3번,207,7759861985,325415,126.925595,37.560027,139628.951613,7.055556,18,3,2호선;공항철도 1호선;경의중앙선,1
571,경의선책거리,474,16051237459,904304,126.928576,37.556110,139248.322581,4.312500,32,3,2호선;공항철도 1호선;경의중앙선,1
580,대흥동주민센터,60,1275357054,94337,126.941252,37.554999,103079.903226,4.447368,38,2,경의중앙선;2호선,2
858,KT구로지사,83,758567017,50924,126.902293,37.481082,103753.306452,4.363636,22,1,2호선,1


In [162]:
middle_market_region = market_region_human_avr.loc[market_region_human_avr['유동인구'] >= 20000]
middle_market_region

,상권_코드_명,점포_수,분기당_매출_금액,분기당_매출_건수,long,lat,유동인구,환승가능한버스노선,500m 버스정류장,환승가능지하철,환승가능 호선명,500m 지하철역
0,이태원 관광특구,1720,102083170176,4441275,126.994377,37.534435,30909.564516,1.904762,21,1,6호선,1
1,명동 남대문 북창동 다동 무교동 관광특구,8631,392306973005,16655824,126.981856,37.564149,138946.870968,9.555556,18,2,1호선;2호선,2
2,동대문패션타운 관광특구,13270,204760984791,6077340,127.011025,37.567312,107866.096774,5.473684,19,4,5호선;4호선;1호선;2호선,2
3,종로?청계 관광특구,5552,172469071750,6280385,126.997374,37.569998,69533.048387,5.160000,25,3,5호선;1호선;2호선,2
5,강남 마이스 관광특구,851,173926103805,4768635,127.060059,37.511003,101897.145161,5.818182,22,2,9호선;2호선,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1638,로데오거리상점가,456,33123581945,1676537,127.127236,37.539043,61812.532258,3.176471,17,2,5호선;8호선,1
1639,성내전통시장,117,3859981574,210781,127.127498,37.533673,30183.629032,1.954545,22,1,5호선,1
1640,암사종합시장,218,11760099134,731306,127.128999,37.550970,24851.935484,1.750000,16,1,8호선,1
1645,명일전통시장,96,3793840824,223786,127.145337,37.549293,27466.016129,3.555556,18,1,5호선,2


# 서울에는 중형상권과 대형상권이 645곳이나 존재한다.
- 그래서 한가지 대안이 필요. 
- 시각화는 유동인구 상위 100곳을 기준으로 하고
- 데이터 비교는 중형상권이상으로 한다.

In [163]:
middle_market_region.to_csv('./2019년 중형상권 이상.csv', encoding='EUC-KR', index=False)
middle_market_region

,상권_코드_명,점포_수,분기당_매출_금액,분기당_매출_건수,long,lat,유동인구,환승가능한버스노선,500m 버스정류장,환승가능지하철,환승가능 호선명,500m 지하철역
0,이태원 관광특구,1720,102083170176,4441275,126.994377,37.534435,30909.564516,1.904762,21,1,6호선,1
1,명동 남대문 북창동 다동 무교동 관광특구,8631,392306973005,16655824,126.981856,37.564149,138946.870968,9.555556,18,2,1호선;2호선,2
2,동대문패션타운 관광특구,13270,204760984791,6077340,127.011025,37.567312,107866.096774,5.473684,19,4,5호선;4호선;1호선;2호선,2
3,종로?청계 관광특구,5552,172469071750,6280385,126.997374,37.569998,69533.048387,5.160000,25,3,5호선;1호선;2호선,2
5,강남 마이스 관광특구,851,173926103805,4768635,127.060059,37.511003,101897.145161,5.818182,22,2,9호선;2호선,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1638,로데오거리상점가,456,33123581945,1676537,127.127236,37.539043,61812.532258,3.176471,17,2,5호선;8호선,1
1639,성내전통시장,117,3859981574,210781,127.127498,37.533673,30183.629032,1.954545,22,1,5호선,1
1640,암사종합시장,218,11760099134,731306,127.128999,37.550970,24851.935484,1.750000,16,1,8호선,1
1645,명일전통시장,96,3793840824,223786,127.145337,37.549293,27466.016129,3.555556,18,1,5호선,2
